<a href="https://colab.research.google.com/github/Edmontong/Bootcamp25_finalproject_Montecarlo_ROIC-based-DCF_valuation_model/blob/main/final_project_bootcamp_montecarlo_dcf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Four factors of forecast

#1. we know the causes of the thing we are trying to forecast
#2. History is somewhat indicative to future
#3. Large amounts of data
#4. forecast does not impact the thing we are trying to forecast

In [ ]:
#forecasting PEPSI CO future freecahsflow

In [ ]:
#In this project, we extract PEPSICO's past 40 years of financial data and insert to a Monte Carlo layer for simulation
#We then take the monte carlo layer result and plug in modern dcf model which uses ROIC*Reinvestment Rate
#for a PEPSICO's valuation which is more accurate and accounts for various scenarios comparing to a traditional, simple DCF

In [ ]:
#First, financials needed for the past 40 years
#Revenue, NOPAT, Reinvestment, ROIC, tax rate, WACC, shares outstanding, market cap, debt
#SALE, EBIT, DP, CAPX, TXT, AT, CHE, ACT, LCT, DLTT, DLC, CSHO, PRCC_F, in WRDS's compustat

In [1]:
pip install wrds

In [16]:
import wrds
import pandas as pd
import numpy as np

In [4]:
db = wrds.Connection()

Enter your WRDS username [root]:edmon
Enter your password:··········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [10]:
pd.set_option('display.max_rows', None)

In [11]:
comp = db.raw_sql("""
    SELECT
        gvkey,
        datadate,
        fyear,
        SALE,
        EBIT,
        DP,
        CAPX,
        TXT,
        AT,
        CHE,
        ACT,
        LCT,
        DLTT,
        DLC,
        CSHO,
        PRCC_F
    FROM comp.funda
    WHERE tic = 'PEP'
      AND indfmt = 'INDL'
      AND datafmt = 'STD'
      AND consol = 'C'
      AND popsrc = 'D'
    ORDER BY datadate;
""")
comp['datadate'] = pd.to_datetime(comp['datadate'])
comp = comp.sort_values('datadate').set_index('datadate')
comp #yearly financial of pepsi the past 75 years, some directly plugged into Monte carlo, some needed for computation of other financial to be plugged in

,gvkey,fyear,sale,ebit,dp,capx,txt,at,che,act,lct,dltt,dlc,csho,prcc_f
datadate,,,,,,,,,,,,,,,
1950-12-31,008479,1950,40.2,2.94,0.93,4.7,1.66,33.4,12.7,19.1,5.4,4.1,0.5,5.752,<NA>
1951-12-31,008479,1951,46.8,3.97,1.5,8.8,1.56,37.0,10.2,18.5,4.8,4.1,0.4,5.752,<NA>
1952-12-31,008479,1952,53.3,7.77,1.8,3.9,3.95,41.2,15.8,23.5,7.0,3.6,0.4,5.752,<NA>
1953-12-31,008479,1953,66.1,11.14,1.82,5.4,5.91,48.6,21.9,29.9,11.9,3.2,0.4,5.752,<NA>
1954-12-31,008479,1954,74.2,12.99,1.89,9.1,6.49,50.8,20.1,29.3,11.2,2.8,0.4,5.813,<NA>
1955-12-31,008479,1955,89.0,18.36,2.06,9.9,9.42,58.1,20.4,32.8,14.3,2.0,0.4,5.909,<NA>
1956-12-31,008479,1956,97.0,17.05,2.63,12.3,9.0,62.7,13.8,32.0,14.6,3.1,0.4,5.919,<NA>
1957-12-31,008479,1957,120.3,18.95,4.26,20.9,10.11,77.4,16.1,33.7,19.4,8.9,3.8,5.926,<NA>
1958-12-31,008479,1958,136.8,24.32,5.3,14.4,11.65,83.9,17.7,36.3,19.0,9.0,1.9,6.139,<NA>


In [14]:
#filling NA with mean
comp = comp.fillna(comp['prcc_f'].mean()) #only prcc_f missing data
comp

,gvkey,fyear,sale,ebit,dp,capx,txt,at,che,act,lct,dltt,dlc,csho,prcc_f
datadate,,,,,,,,,,,,,,,
1950-12-31,008479,1950,40.2,2.94,0.93,4.7,1.66,33.4,12.7,19.1,5.4,4.1,0.5,5.752,65.10416
1951-12-31,008479,1951,46.8,3.97,1.5,8.8,1.56,37.0,10.2,18.5,4.8,4.1,0.4,5.752,65.10416
1952-12-31,008479,1952,53.3,7.77,1.8,3.9,3.95,41.2,15.8,23.5,7.0,3.6,0.4,5.752,65.10416
1953-12-31,008479,1953,66.1,11.14,1.82,5.4,5.91,48.6,21.9,29.9,11.9,3.2,0.4,5.752,65.10416
1954-12-31,008479,1954,74.2,12.99,1.89,9.1,6.49,50.8,20.1,29.3,11.2,2.8,0.4,5.813,65.10416
1955-12-31,008479,1955,89.0,18.36,2.06,9.9,9.42,58.1,20.4,32.8,14.3,2.0,0.4,5.909,65.10416
1956-12-31,008479,1956,97.0,17.05,2.63,12.3,9.0,62.7,13.8,32.0,14.6,3.1,0.4,5.919,65.10416
1957-12-31,008479,1957,120.3,18.95,4.26,20.9,10.11,77.4,16.1,33.7,19.4,8.9,3.8,5.926,65.10416
1958-12-31,008479,1958,136.8,24.32,5.3,14.4,11.65,83.9,17.7,36.3,19.0,9.0,1.9,6.139,65.10416


In [18]:
comp.info() #all necessary metrics are float

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 75 entries, 1950-12-31 to 2024-12-31
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   gvkey   75 non-null     string 
 1   fyear   75 non-null     Int64  
 2   sale    75 non-null     Float64
 3   ebit    75 non-null     Float64
 4   dp      75 non-null     Float64
 5   capx    75 non-null     Float64
 6   txt     75 non-null     Float64
 7   at      75 non-null     Float64
 8   che     75 non-null     Float64
 9   act     75 non-null     Float64
 10  lct     75 non-null     Float64
 11  dltt    75 non-null     Float64
 12  dlc     75 non-null     Float64
 13  csho    75 non-null     Float64
 14  prcc_f  75 non-null     Float64
dtypes: Float64(13), Int64(1), string(1)
memory usage: 10.4 KB


In [21]:
#computation for other financial using a portion of the dataframe's data
# EBIT margin
comp['ebit_margin'] = comp['ebit'] / comp['sale']
# revenue growth
comp['rev_growth'] = comp['sale'].pct_change()
# raw effective tax rate = taxes / EBIT (avoid div by 0)
comp['tax_rate'] = comp['txt'] / comp['ebit'].replace(0, np.nan)
# NOPAT = EBIT * (1 - tax rate)
comp['nopat'] = comp['ebit'] * (1 - comp['tax_rate'])
# invested capital: total assets - cash
comp['invested_capital'] = comp['at'] - comp['che']
# ROIC = NOPAT_t / IC
comp['roic'] = comp['nopat'] / comp['invested_capital']
# Net working capital
comp['nwc'] = comp['act'] - comp['lct']
# Change in NWC
comp['change_nwc'] = comp['nwc'] - comp['nwc'].shift(1)
# Reinvestment = Capex - Depreciation + change in NWC
comp['reinvestment'] = comp['capx'] - comp['dp'] + comp['change_nwc']
# Reinvestment rate RR = Reinvestment / NOPAT
comp['rr'] = comp['reinvestment'] / comp['nopat']
#financial to be plugged in Monte carlo
driver_cols = [
    'fyear', 'sale', 'ebit', 'ebit_margin', 'rev_growth',
    'tax_rate', 'nopat', 'invested_capital', 'roic',
    'reinvestment', 'rr']
drivers = comp[driver_cols].dropna()
drivers


,fyear,sale,ebit,ebit_margin,rev_growth,tax_rate,nopat,invested_capital,roic,reinvestment,rr
datadate,,,,,,,,,,,
1951-12-31,1951,46.8,3.97,0.084829,0.164179,0.392947,2.41,26.8,0.089925,7.3,3.029046
1952-12-31,1952,53.3,7.77,0.145779,0.138889,0.508366,3.82,25.4,0.150394,4.9,1.282723
1953-12-31,1953,66.1,11.14,0.168533,0.24015,0.530521,5.23,26.7,0.19588,5.08,0.971319
1954-12-31,1954,74.2,12.99,0.175067,0.122542,0.499615,6.5,30.7,0.211726,7.31,1.124615
1955-12-31,1955,89.0,18.36,0.206292,0.199461,0.513072,8.94,37.7,0.237135,8.24,0.9217
1956-12-31,1956,97.0,17.05,0.175773,0.089888,0.527859,8.05,48.9,0.164622,8.57,1.064596
1957-12-31,1957,120.3,18.95,0.157523,0.240206,0.533509,8.84,61.3,0.144209,13.54,1.531674
1958-12-31,1958,136.8,24.32,0.177778,0.137157,0.47903,12.67,66.2,0.19139,12.1,0.955012
1959-12-31,1959,157.8,30.37,0.192459,0.153509,0.488311,15.54,69.4,0.223919,24.61,1.583655


In [ ]:
#Second, code for Monte Carlo Simulation

In [22]:
def monte_carlo_dcf(
    drivers,
    wacc,
    net_debt=0.0,
    shares_out=1.0,
    horizon=10,
    n_sims=5000, #5000 simulations
    g_term=0.02, #assuming 2% terminal growth rate
    seed=42):

    np.random.seed(seed)
    #1. setting up
    #Mean and std of data in drivers df
    mu_g, sigma_g = drivers['rev_growth'].mean(), drivers['rev_growth'].std()
    mu_m, sigma_m  = drivers['ebit_margin'].mean(), drivers['ebit_margin'].std()
    mu_roic, sigma_roic = drivers['roic'].mean(), drivers['roic'].std()
    mu_rr, sigma_rr = drivers['rr'].mean(), drivers['rr'].std()

    #use median tax rate
    tax_rate = drivers['tax_rate'].median()

    # take last historical revenue for starting revenue in Monte Carlo forecast
    revenue_0 = drivers['sale'].iloc[-1]



    #2. Monte carlo fcf forecast
    ev_results = []
    for _ in range(n_sims):
        revenue = revenue_0

        fcff_list = []
        last_nopat = None
        last_roic = mu_roic  # take ROIC mean as initial ROIC level for MontecarloForecast

        for t in range(1, horizon + 1):

            #random drivers for year t
            g_t = np.random.normal(mu_g,    sigma_g)
            m_t = np.random.normal(mu_m,    sigma_m)
            roic_t = np.random.normal(mu_roic, sigma_roic)
            rr_t = np.random.normal(mu_rr,   sigma_rr)

            # update revenue
            revenue = revenue * (1 + g_t)

            # EBIT and NOPAT
            ebit_t  = revenue * m_t
            nopat_t = ebit_t * (1 - tax_rate)

            # FCFF = NOPAT * (1 - RR)
            fcff_t = nopat_t * (1 - rr_t)

            fcff_list.append(fcff_t)
            last_nopat = nopat_t
            last_roic  = roic_t




        # 3. Terminal value calculation
        # terminal reinvestment rate calculated by g_term and last_roic
        rr_term = g_term / last_roic
        fcff_T1 = last_nopat * (1 + g_term) * (1 - rr_term)
        tv = fcff_T1 / (wacc - g_term) #Terminal Value



        #4. complete ev Valuation = sum of fcfs + TV
        years = np.arange(1, horizon + 1)
        cashflows = np.array(fcff_list[:-1] + [fcff_list[-1] + tv])

        ev = np.sum(cashflows / (1 + wacc) ** years)
        ev_results.append(ev)

    ev_results = np.array(ev_results)

    # 5. EV converted to equity value and per-share price
    equity_values = ev_results - net_debt
    prices = equity_values / shares_out

    return {
        "ev": ev_results,
        "equity": equity_values,
        "price": prices,
        "params": {
            "mu_g": mu_g, "sigma_g": sigma_g,
            "mu_margin": mu_m, "sigma_margin": sigma_m,
            "mu_roic": mu_roic, "sigma_roic": sigma_roic,
            "mu_rr": mu_rr, "sigma_rr": sigma_rr,
            "tax_rate": tax_rate,
            "revenue_0": revenue_0
        }
    }


In [ ]:
#Third, testing the model

In [26]:
#Compute net debt and shares_out from latest year, setting up for equity value and equity value per share calculation
latest = comp.iloc[-1]
# Net debt = longterm debt + short term debt - cash
net_debt = (latest['dltt'] + latest['dlc']) - latest['che']   # in millions
# Shares outstanding in millions
shares_out = latest['csho']


# Run Monte Carlo DCF model
wacc = 0.08  # assuming 8% wacc

results = monte_carlo_dcf(
    drivers=drivers,
    wacc=wacc,
    net_debt=net_debt,
    shares_out=shares_out,
    horizon=10,
    n_sims=5000,
    g_term=0.02,
    seed=42)

equity_value = results["ev"] #Equity value
prices = results["price"]  # per-share price

print(f"Mean EV (millions):   {equity_value.mean():,.2f}") #until 2 decimals
print(f"Median EV (millions): {np.median(equity_value):,.2f}")
print(f"Mean price ($/share): {prices.mean():.2f}")
print(f"Median price ($/share): {np.median(prices):.2f}")

#below are our monte carlo ROIC-based DCF estimates
#in comparison to current valuation: EV = 203.79 billion/ 203,790 millions; Per share price = 149.04 $

Mean EV (millions):   208,561.00
Median EV (millions): 190,324.12
Mean price ($/share): 123.96
Median price ($/share): 110.67


In [ ]:
#in comparison, simple DCF which assumes constant FCF change rate

In [27]:
def simple_constant_growth_dcf(
    drivers,
    wacc,
    net_debt,
    shares_out,
    growth=0.03, # assumed constant fcf growth rate of 3%
    g_term=0.02, # terminal growth
    horizon=10
):
    # 1. Last historical FCF from drivers dataset
    fcff_0 = (drivers['nopat'] * (1 - drivers['rr'])).iloc[-1]

    # 2. Forecast FCFF
    fcff_list = []
    for t in range(1, horizon + 1):
        fcff_t = fcff_0 * (1 + growth) ** t
        fcff_list.append(fcff_t)

    # 3. Terminal value
    fcff_T1 = fcff_0 * (1 + growth) ** (horizon + 1)
    tv = fcff_T1 / (wacc - g_term)

    # 4. Valuation = Discount all free cashflows + TV
    years = np.arange(1, horizon + 1)
    cashflows = np.array(fcff_list[:-1] + [fcff_list[-1] + tv])
    ev = np.sum(cashflows / (1 + wacc) ** years)   # enterprise value (millions)

    # 5. Convert EV to Equity value & Price per share
    equity = ev - net_debt          # equity value (millions)
    price = equity / shares_out     # price in USD (shares_out is in millions)

    return {
        "EV_millions": ev,
        "Equity_millions": equity,
        "Price_per_share": price,
        "Assumptions": {
            "FCFF_0": fcff_0,
            "growth": growth,
            "terminal_growth": g_term,
            "wacc": wacc}}


In [29]:
latest = comp.iloc[-1] #latest financial data
net_debt   = (latest['dltt'] + latest['dlc']) - latest['che']
shares_out = latest['csho']
wacc = 0.08   # same WACC used in Monte Carlo
growth = 0.03
g_term = 0.02

simple = simple_constant_growth_dcf(
    drivers=drivers,
    wacc=wacc,
    net_debt=net_debt,
    shares_out=shares_out,
    growth=growth,
    g_term=g_term,
    horizon=10)
print(f"Equity value (millions):     {simple['Equity_millions']:,.2f}")
print(f"price ($/share):   {simple['Price_per_share']:.2f}")
#In comparison to a simple DCF model, the simple models forecasts PEPSICO's equity value 162.74 billion, and price per share 118.62$

Equity value (millions):     162,740.76
price ($/share):   118.62
